# 🔄 JDBC: Manejo de Transacciones y Puntos de Restauración


## 1. Introducción

En este cuadernillo aprenderás a trabajar con transacciones en JDBC, una técnica esencial para garantizar la consistencia de los datos cuando se ejecutan múltiples operaciones relacionadas.

También veremos cómo utilizar `Savepoint` para controlar partes específicas dentro de una transacción.



## 2. ¿Qué es una Transacción?

Una transacción agrupa varias operaciones SQL para que se ejecuten como una sola unidad. Si alguna falla, todas se revierten.

📌 Propiedades ACID:
- **Atomicidad**: todo o nada.
- **Consistencia**: los datos cumplen todas las reglas del sistema.
- **Aislamiento**: las transacciones no interfieren entre sí.
- **Durabilidad**: los cambios confirmados son permanentes.



## 3. Transacciones en JDBC

Por defecto, JDBC usa autocommit (`true`): cada sentencia se confirma automáticamente. Para agrupar operaciones:

```java
Connection conexion = DriverManager.getConnection(url, usuario, contraseña);
conexion.setAutoCommit(false); // Desactiva auto commit

try {
    PreparedStatement ps1 = conexion.prepareStatement("UPDATE cuentas SET saldo = saldo - ? WHERE id = ?");
    ps1.setDouble(1, 100);
    ps1.setInt(2, 1);
    ps1.executeUpdate();

    PreparedStatement ps2 = conexion.prepareStatement("UPDATE cuentas SET saldo = saldo + ? WHERE id = ?");
    ps2.setDouble(1, 100);
    ps2.setInt(2, 2);
    ps2.executeUpdate();

    conexion.commit(); // Confirma si todo va bien
    System.out.println("Transferencia realizada correctamente.");
} catch (SQLException e) {
    conexion.rollback(); // Revierte si hay error
    System.out.println("Error en la transacción: " + e.getMessage());
} finally {
    conexion.setAutoCommit(true); // Vuelve al modo automático
}
```



## 4. Puntos de Restauración (Savepoints)

Permiten deshacer parte de una transacción sin afectar todo el bloque.

```java
conexion.setAutoCommit(false);

try {
    Statement stmt = conexion.createStatement();
    stmt.executeUpdate("INSERT INTO alumnos VALUES (10, 'Marta', 20, 'ASIR1')");
    Savepoint punto1 = conexion.setSavepoint();

    stmt.executeUpdate("INSERT INTO alumnos VALUES (11, 'Carlos', 18, 'DAM2')");
    stmt.executeUpdate("INSERT INTO alumnos VALUES (12, 'Error', 'texto_invalido', 'DAM2')"); // Falla

    conexion.commit();
} catch (SQLException e) {
    System.out.println("Error detectado. Restaurando al punto de control...");
    try {
        conexion.rollback(punto1); // Reversión parcial
        conexion.commit();         // Confirma lo válido
    } catch (SQLException ex) {
        System.out.println("Error al hacer rollback parcial: " + ex.getMessage());
    }
} finally {
    conexion.setAutoCommit(true);
}
```



## 5. Buenas Prácticas al Usar Transacciones

✅ Usa bloques `try-catch-finally`.  
✅ Desactiva `autoCommit` solo cuando sea necesario.  
✅ Confirma con `commit()` solo si todo se ejecutó correctamente.  
✅ Usa `rollback()` ante errores.  
✅ Usa `savepoint` para mantener operaciones válidas en errores parciales.  
✅ Siempre reactiva `autoCommit` al final.  
✅ Separa lógica de negocio y acceso a base de datos.
